# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

1. 데이터 전처리
2. 상관분석 함수
3. activedefrag 있는 경우
4. 전체 knob
5. activedefrag 없는 경우
6. pointbeir~

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# internal metrics 분류 결과
cluster0 =[ 
    'uptime_in_seconds',
    'hz',
    'lru_clock',
    'used_memory_peak',
    'maxmemory',
    'maxmemory_human',
    'rdb_last_save_time',
    'total_net_output_bytes',
    'used_cpu_sys',
    'used_cpu_user',
    'allocator_allocated',
    'allocator_active',
    'allocator_frag_bytes',
    'instantaneous_ops_per_sec',
    'instantaneous_input_kbps',
    'instantaneous_output_kbps',
    'allocator_resident',
    'allocator_rss_ratio',
    'allocator_rss_bytes' 
]

cluster1 = [
    'used_memory_rss',
    'used_memory_rss_human',
    'rss_overhead_ratio',
    'rss_overhead_bytes',
    'mem_fragmentation_ratio',
    'mem_fragmentation_bytes'
]




knob_list = [
    "save0_0",
    "save0_1",
    "save1_0",
    "save1_1",
    "save2_0",
    "save2_1",

    "rdbcompression",
    "rdbchecksum",
    "rdb-save-incremental-fsync",

    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",
    
    "maxmemory-policy",
    "maxmemory-samples",
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del",

    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",
    "hz",
    "dynamic-hz"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [4]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
# internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

file_dir = '../data/'
file_name = 'internal_metrics_6000_6.csv'

internal_csv = pd.read_csv(file_dir+file_name)

In [5]:
# 데이터 불러오기 knob_list
# knobs = pd.read_csv('./data/result_config0.csv')

file_dir = '../data/'
file_name = 'result_config_6000_6.csv'

knobs = pd.read_csv(file_dir + file_name)

In [6]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

## 데이터 전처리

In [7]:
# 클러스터 합치기
cluster = cluster0+cluster1

In [8]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

In [9]:
# throughput 결측치 제거
total_internal_df.dropna()

,uptime_in_seconds,hz,lru_clock,used_memory_peak,maxmemory,maxmemory_human,rdb_last_save_time,total_net_output_bytes,used_cpu_sys,used_cpu_user,...,instantaneous_output_kbps,allocator_resident,allocator_rss_ratio,allocator_rss_bytes,used_memory_rss,used_memory_rss_human,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes
0,22,7,1745211,38704616,2831155200,2.64,1629135141,5048972,8.506465,3.861151,...,226.38,42209280,1.08,3276800,42033152,40.09,1.00,-176128,1.09,3410544
1,21,32,1745245,38704616,2306867200,2.15,1629135176,5048972,8.464944,3.926858,...,275.94,44310528,1.14,5378048,42037248,40.09,0.95,-2273280,1.09,3414640
2,21,40,1745272,38704616,1782579200,1.66,1629135203,5048972,8.397651,3.828146,...,252.57,42209280,1.08,3276800,42029056,40.08,1.00,-180224,1.09,3406448
3,21,37,1745300,38704616,1048576000,1000.00,1629135231,5048972,8.392912,3.816183,...,258.74,42209280,1.08,3276800,41828352,39.89,0.99,-380928,1.08,3205744
4,22,30,1745328,38704616,1992294400,1.86,1629135258,5049038,9.027816,4.059768,...,266.38,42209280,1.08,3276800,42037248,40.09,1.00,-172032,1.09,3414640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,22,25,1823694,38704616,2936012800,2.73,1629213624,5048972,8.913044,4.146845,...,294.26,42160128,1.08,3186688,41787392,39.85,0.99,-372736,1.08,3126224
2496,22,7,1823722,38704616,1572864000,1.46,1629213652,5049038,8.420312,3.992813,...,209.28,44310528,1.14,5378048,42037248,40.09,0.95,-2273280,1.09,3414640
2497,22,13,1823750,38704616,3040870400,2.83,1629213680,5049038,8.737909,4.083693,...,184.97,42209280,1.08,3276800,42041344,40.09,1.00,-167936,1.09,3418736
2498,22,35,1823779,38704632,2306867200,2.15,1629213709,5048972,8.581871,3.811128,...,108.87,44310528,1.14,5378048,42037248,40.09,0.95,-2273280,1.09,3414640


In [9]:
# internal_csv 결측치 확인 -> 없음!
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   uptime_in_seconds          2500 non-null   int64  
 1   hz                         2500 non-null   int64  
 2   lru_clock                  2500 non-null   int64  
 3   used_memory_peak           2500 non-null   int64  
 4   maxmemory                  2500 non-null   int64  
 5   maxmemory_human            2500 non-null   float64
 6   rdb_last_save_time         2500 non-null   int64  
 7   total_net_output_bytes     2500 non-null   int64  
 8   used_cpu_sys               2500 non-null   float64
 9   used_cpu_user              2500 non-null   float64
 10  allocator_allocated        2500 non-null   int64  
 11  allocator_active           2500 non-null   int64  
 12  allocator_frag_bytes       2500 non-null   int64  
 13  instantaneous_ops_per_sec  2500 non-null   int64

In [10]:
# maxmemory 처리 throughtput은 처리할 필요없다.
total_internal_df['maxmemory']

0       2831155200
1       2306867200
2       1782579200
3       1048576000
4       1992294400
           ...    
2495    2936012800
2496    1572864000
2497    3040870400
2498    2306867200
2499    1468006400
Name: maxmemory, Length: 2500, dtype: int64

In [11]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [12]:
# knob_list 만 추출, AOF 관련 제외
subTotal_knobs = extract_data(knob_list, knobs)

In [13]:
subTotal_knobs

,save0_0,save0_1,save1_0,save1_1,save2_0,save2_1,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,...,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,805,5,178,91,36,9230,no,no,no,yes,...,volatile-lru,5,no,yes,yes,258,253,no,7,yes
1,1097,3,499,33,82,9410,yes,no,no,yes,...,allkeys-random,3,yes,yes,yes,360,226,yes,32,yes
2,1018,2,497,23,36,8649,yes,no,yes,yes,...,allkeys-lfu,3,yes,yes,yes,364,136,yes,40,no
3,858,5,423,14,80,8407,no,no,no,yes,...,noeviction,3,no,yes,no,417,173,no,37,no
4,727,6,230,54,61,8327,yes,no,no,yes,...,allkeys-random,4,no,yes,no,522,141,yes,30,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,726,6,258,25,54,8545,no,no,no,yes,...,allkeys-random,7,yes,yes,yes,538,54,yes,25,no
2496,930,8,166,98,50,10113,no,no,yes,yes,...,allkeys-lru,4,yes,no,yes,735,67,no,7,yes
2497,825,4,436,69,43,10806,no,no,no,yes,...,allkeys-random,4,no,no,yes,538,63,no,13,no
2498,1025,1,122,50,67,9759,yes,no,no,yes,...,volatile-random,3,no,yes,no,337,54,no,35,yes


In [14]:
#activedefrag knobs 분리

# activedefrag O 존재
subTotal_knobs_Active = subTotal_knobs.dropna()
Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# activedefrag X 존재하지 않는 나머지 반
subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# activedefrag O and X 전체 
subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
total_index = list(subTotal_knobs_T.index)   # 인덱스

In [15]:
# knobs에서 타입별로 분리 (수치, 불리언, 불리언아닌 문자)

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

# 수치, 불리언, 불리언아닌것 (전체)
numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

# 수치, 불리언, 불리언아닌것 (activedefrag)
numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

# 수치, 불리언, 불리언아닌것 (activedefrag 없는것)
numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [16]:
# activedefrag O
total_internal_Active = total_internal_df.iloc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.iloc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.iloc[total_index]

---

## 상관분석 함수

In [17]:
def get_outlier(df=None, column=None, weight=1.5):
    
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [18]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


# knob하나 internal 여러개 상관계수 구하기 이상치 제거 버전
# pearson
## 모수 검정
def cal_pearsonr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pearsonr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = kendalltau(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

    
# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = spearmanr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])



In [19]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

def extract_pvalue_outlier(knobs, internal_df):
    print('pearson')
    cal_pearsonr_outlier(knobs, internal_df)
    print()
    
    print('kendalltau')
    cal_kendalltau_outlier(knobs, internal_df)
    print()
    
    print('spearman')
    cal_spearmanr_outlier(knobs, internal_df)
    print()
    

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)
        
        
# knob 리스트 각각 별로 extract_pvalue 수행 이상치 제거 
def extract_pvalue_knob_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue_outlier(subTotal_knobs[knob], total_internal)

---

## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [20]:
numericalActive_var
numericalActive_var_new = numericalActive_var[6:10]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [21]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
                         corr   p-value
allocator_resident   0.055844  0.048207
allocator_rss_ratio  0.057253  0.042821
allocator_rss_bytes  0.056016  0.047520

kendalltau
                        corr  p-value
rss_overhead_ratio -0.051988  0.02251

spearman
                        corr   p-value
rss_overhead_ratio -0.064179  0.023149

###active-defrag-threshold-upper###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-min###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-max###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns

In [23]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    active_result['cluster'+str(idx)] = cluster_val

In [24]:
active_result

,cluster0,cluster1
active-defrag-threshold-lower,0.026358,0.017907
active-defrag-threshold-upper,0.023418,0.035128
active-defrag-cycle-min,0.023584,0.023919
active-defrag-cycle-max,0.015026,0.010496


In [30]:
# active_result

### 이상치 제거 후 상관계수

In [53]:
extract_pvalue_knob_outlier(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson

kendalltau

spearman

###active-defrag-threshold-upper###
pearson

kendalltau

spearman

###active-defrag-cycle-min###
pearson

kendalltau

spearman

###active-defrag-cycle-max###
pearson

kendalltau

spearman



---

## 전체 knob
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [31]:
numerical_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [32]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                            corr   p-value
mem_fragmentation_ratio -0.03388  0.036831

spearman
                             corr   p-value
mem_fragmentation_ratio -0.041749  0.036861

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                             corr   p-value
mem_fragmentation_ratio  0.032439  0.045625
allocator_resident       0.037987  0.015417

spearman
                             corr   p-value
mem_fragmentation_ratio  0.040057  0.045217
allocator_resident       0.048404  0.015503

###save1_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [

In [33]:
subTotal_knobs_T  # knobs에서 aof 관련 제거한 것
total_internal_T  # 전체

,used_memory_rss,used_memory_rss_human,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes,uptime_in_seconds,hz,lru_clock,used_memory_peak,...,used_cpu_user,allocator_allocated,allocator_active,allocator_frag_bytes,instantaneous_ops_per_sec,instantaneous_input_kbps,instantaneous_output_kbps,allocator_resident,allocator_rss_ratio,allocator_rss_bytes
0,42033152,40.09,1.00,-176128,1.09,3410544,22,7,1745211,38704616,...,3.861151,38658184,38932480,274296,45654,3187.77,226.38,42209280,1.08,3276800
1,42037248,40.09,0.95,-2273280,1.09,3414640,21,32,1745245,38704616,...,3.926858,38658328,38932480,274152,55643,3885.23,275.94,44310528,1.14,5378048
2,42029056,40.08,1.00,-180224,1.09,3406448,21,40,1745272,38704616,...,3.828146,38657784,38932480,274696,50950,3557.60,252.57,42209280,1.08,3276800
3,41828352,39.89,0.99,-380928,1.08,3205744,21,37,1745300,38704616,...,3.816183,38658184,38932480,274296,52136,3640.36,258.74,42209280,1.08,3276800
4,42037248,40.09,1.00,-172032,1.09,3414640,22,30,1745328,38704616,...,4.059768,38657928,38932480,274552,53721,3751.05,266.38,42209280,1.08,3276800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,41787392,39.85,0.99,-372736,1.08,3126224,22,25,1823694,38704616,...,4.146845,38696888,38973440,276552,59397,4147.40,294.26,42160128,1.08,3186688
2496,42037248,40.09,0.95,-2273280,1.09,3414640,22,7,1823722,38704616,...,3.992813,38658000,38932480,274480,42192,2946.03,209.28,44310528,1.14,5378048
2497,42041344,40.09,1.00,-167936,1.09,3418736,22,13,1823750,38704616,...,4.083693,38658336,38932480,274144,37208,2598.02,184.97,42209280,1.08,3276800
2498,42037248,40.09,0.95,-2273280,1.09,3414640,22,35,1823779,38704632,...,3.811128,38658328,38932480,274152,21943,1532.13,108.87,44310528,1.14,5378048


In [25]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    numerical_result['cluster'+str(idx)] = cluster_val

In [26]:
numerical_result

,cluster0,cluster1
save0_0,0.010876,0.021846
save0_1,0.013192,0.019943
save1_0,0.015842,0.020149
save1_1,0.011183,0.017930
save2_0,0.014607,0.011214
save2_1,0.015264,0.023353
maxmemory,0.096824,0.008709
maxmemory-samples,0.013840,0.006326
hash-max-ziplist-entries,0.009534,0.021681
hash-max-ziplist-value,0.012926,0.016140


In [36]:
numerical_result

,cluster0,cluster1,cluster2,cluster3
save0_0,0.021846,0.008927,0.014598,0.009929
save0_1,0.019943,0.014847,0.007956,0.018147
save1_0,0.020149,0.013239,0.010465,0.035270
save1_1,0.017930,0.008781,0.013621,0.014315
save2_0,0.011214,0.017536,0.013853,0.006355
save2_1,0.023353,0.022299,0.008478,0.005386
maxmemory,0.008709,0.180528,0.004549,0.002359
maxmemory-samples,0.006326,0.018330,0.010886,0.004782
hash-max-ziplist-entries,0.021681,0.006836,0.009750,0.018099
hash-max-ziplist-value,0.016140,0.012104,0.006438,0.028639


In [54]:
extract_pvalue_knob_outlier(numerical_var, subTotal_knobs_T,total_internal_T)

###save0_0###
pearson

kendalltau

spearman

###save0_1###
pearson

kendalltau

spearman

###save1_0###
pearson

kendalltau

spearman

###save1_1###
pearson

kendalltau

spearman

###save2_0###
pearson

kendalltau

spearman

###save2_1###
pearson
rss_overhead_bytes
-0.046419563769345834

kendalltau
used_memory_rss
-0.02924722138006238
rss_overhead_bytes
-0.03504107721117922

spearman
used_memory_rss
-0.04192725341960683
rss_overhead_bytes
-0.05035345730583921

###maxmemory###
pearson
maxmemory
0.9999999999999998
maxmemory_human
0.999985936351003

kendalltau
maxmemory
0.9999999999999999
maxmemory_human
1.0

spearman
maxmemory
1.0
maxmemory_human
1.0

###maxmemory-samples###
pearson

kendalltau

spearman

###hash-max-ziplist-entries###
pearson

kendalltau

spearman

###hash-max-ziplist-value###
pearson

kendalltau

spearman

###hz###
pearson
used_memory_rss
-0.15780303088576614
used_memory_rss_human
-0.1540371620286659
rss_overhead_bytes
-0.13910118085511436
hz
1.0
lru_clock
0.0422212842

---

## Activedefrag 없는 knob들
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [37]:
numericalActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [38]:
extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
        corr   p-value
hz  0.037937  0.047294

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
                        corr   p-value
lru_clock           0.059351  0.035746
rdb_last_save_time  0.059352  0.035743

kendalltau
                           corr   p-value
used_memory_rss       -0.044470  0.030427
used_memory_rss_human -0.047318  0.027262
lru_clock              0.040269  0.042872
rdb_last_save_time     0.040269  0.042872

spearman
                           corr   p-value
used_memory_rss       -0.060946  0.031056
used_memory_rss_human -0.062604  0.026751
lru_clock              0.057114  0.043329
rdb_last_save_time     0.057114  0.043329

###save1_0###
pearson
                        corr   p-value
lru_clock          -0.070523  0.012561
rdb_last_save_time -0.070522  0.012563
used_cpu_sys       -0.061330  0.030011

kendalltau
                        corr   p-va

## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [39]:
numericalNoActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [40]:
extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_0###
pearson
                         corr   p-value
lru_clock            0.058587  0.038509
rdb_last_save_time   0.058689  0.038169
allocator_resident   0.059750  0.034810
allocator_rss_ratio  0.058408  0.039104
allocator_rss_bytes  0.059935  0.034250

kendalltau
                             corr   p-value
mem_fragmentation_ratio  0.051463  0.025128
lru_clock                0.039195  0.038311
rdb_last_save_time       0.039195  0.038311
allocator_resident       0.053478  0.014550
allocator_rss_ratio      0.046319  0.045418
allocator_rss_bytes      0.047804  0.024630

spearman
            

---

## pointbiserialr 분석

In [27]:
from scipy.stats import pointbiserialr
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

    
# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


# 이상치 제거 버전
def cal_pointbiserialr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pointbiserialr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue_outlier(knobs, internal_df):
    print('pointbierialr')
    cal_pointbiserialr(knobs, internal_df)
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [28]:
boolean_var

['rdbcompression',
 'rdbchecksum',
 'rdb-save-incremental-fsync',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del',
 'activerehashing',
 'dynamic-hz']

In [29]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [44]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                            corr   p-value
total_net_output_bytes -0.056501  0.004715

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
                         corr   p-value
rss_overhead_ratio  -0.046519  0.020017
rss_overhead_bytes  -0.048922  0.014431
allocator_resident   0.043003  0.031551
allocator_rss_ratio  0.043123  0.031080
allocator_rss_bytes  0.043044  0.031389

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
        corr   p-value
hz -0.045644  0.022474

###activerehashing###
pointbierialr
                         corr   p-value
allocator_allocated -0.044451  0.026248
allocator_active    -0.040469  0.043047

###dynamic-hz###
pointbierialr
                     corr   p-value
ma

In [30]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    bool_result['cluster'+str(idx)] = cluster_val

In [31]:
bool_result

,cluster0,cluster1
rdbcompression,0.020921,0.007174
rdbchecksum,0.010391,0.005528
rdb-save-incremental-fsync,0.012075,0.022363
lazyfree-lazy-eviction,0.013908,0.031083
lazyfree-lazy-expire,0.012880,0.020375
lazyfree-lazy-server-del,0.015724,0.012797
activerehashing,0.023691,0.021916
dynamic-hz,0.019657,0.014893


In [314]:
bool_result

,cluster0,cluster1,cluster2,cluster3
rdbcompression,0.043045,0.247608,0.132567,0.017334
rdbchecksum,0.033166,0.112314,0.049670,0.035439
rdb-save-incremental-fsync,0.134179,0.094039,0.083578,0.051800
lazyfree-lazy-eviction,0.186498,0.111021,0.024066,0.129169
lazyfree-lazy-expire,0.122251,0.127492,0.093513,0.023721
lazyfree-lazy-server-del,0.076782,0.175183,0.051966,0.071605
activerehashing,0.131495,0.178660,0.191599,0.079875
dynamic-hz,0.089357,0.175908,0.114635,0.082941


In [57]:
#이상치 제거
extract_pvalue_knob_point_outlier(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                            corr   p-value
total_net_output_bytes -0.056501  0.004715

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
                         corr   p-value
rss_overhead_ratio  -0.046519  0.020017
rss_overhead_bytes  -0.048922  0.014431
allocator_resident   0.043003  0.031551
allocator_rss_ratio  0.043123  0.031080
allocator_rss_bytes  0.043044  0.031389

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
        corr   p-value
hz -0.045644  0.022474

###activerehashing###
pointbierialr
                         corr   p-value
allocator_allocated -0.044451  0.026248
allocator_active    -0.040469  0.043047

###dynamic-hz###
pointbierialr
                     corr   p-value
ma

---

In [ ]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)